In [2]:
import json
import numpy as np
import os

import subprocess
import glob
from soma import aims

from sulci.registration.spam import spam_register

import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

from soma.aimsalgo import MorphoGreyLevel_S16

# Global static variables
_AIMS_BINARY_ONE = 32767
_dilation = 1
_threshold = 0

# launching anatomist
a = ana.Anatomist()

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
existing QApplication: 0
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jc225751'


create qapp
done
Starting Anatomist.....
config file : /casa/home/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module meshsplit
loading module palettecontrols
loading module volumepalettes
loading module save_resampled
loading module profilewindow
loading module statsplotwindow
loading module anacontrolmenu
loading module infowindow
loading module foldsplit
loading module simple_controls
loading module histogram
loading module valuesplotwindow
loading module ana_image_math
loading module modelGraphs
loading module paletteViewer
loading module bsa_proba
loading module gradientpalette
loading module selection
loading module gltf_io
all python modules loaded
Anatomist started.


In [3]:
def dilate(mask, radius=_dilation):
    """Makes a dilation radius _dilation, in mm
    """
    arr = mask.np
    # Binarization of mask
    arr[arr < 1] = 0
    arr[arr >= 1] = _AIMS_BINARY_ONE
    # Dilates initial volume of 10 mm
    morpho = MorphoGreyLevel_S16()
    dilate = morpho.doDilation(mask, radius)
    arr_dilate = dilate.np
    arr_dilate[arr_dilate >= 1] = 1
    return dilate

In [4]:
spam_file = '/neurospin/dico/data/deep_folding/current/mask/2mm/regions/L/Sc.Cal.-S.Li._left.nii.gz'
skel_path = '/neurospin/dico/data/deep_folding/current/datasets/pclean/binarized_skeletons/L'
skel_files = glob.glob(f'{skel_path}/*.nii.gz')
skel_files

['/neurospin/dico/data/deep_folding/current/datasets/pclean/binarized_skeletons/L/Lbinarized_skeleton_hades.nii.gz',
 '/neurospin/dico/data/deep_folding/current/datasets/pclean/binarized_skeletons/L/Lbinarized_skeleton_sujet04.nii.gz',
 '/neurospin/dico/data/deep_folding/current/datasets/pclean/binarized_skeletons/L/Lbinarized_skeleton_icbm200T.nii.gz',
 '/neurospin/dico/data/deep_folding/current/datasets/pclean/binarized_skeletons/L/Lbinarized_skeleton_shiva.nii.gz',
 '/neurospin/dico/data/deep_folding/current/datasets/pclean/binarized_skeletons/L/Lbinarized_skeleton_hyperion.nii.gz',
 '/neurospin/dico/data/deep_folding/current/datasets/pclean/binarized_skeletons/L/Lbinarized_skeleton_icbm300T.nii.gz',
 '/neurospin/dico/data/deep_folding/current/datasets/pclean/binarized_skeletons/L/Lbinarized_skeleton_icbm100T.nii.gz',
 '/neurospin/dico/data/deep_folding/current/datasets/pclean/binarized_skeletons/L/Lbinarized_skeleton_s12590.nii.gz',
 '/neurospin/dico/data/deep_folding/current/datas

In [5]:
mask_result = aims.read(spam_file)
# Makes binarization and dilation on spam
mask_result[mask_result.np <= _threshold] = 0
mask_result.np[:] = dilate(mask_result).np

# Reads initial spam volume
spam_vol = aims.read(spam_file, dtype="Volume_FLOAT")
spam_vol.np[:] = spam_vol.np
spam_vol.np[:] = spam_vol.np/61

# aims.write(spam_vol, "/tmp/spam.nii.gz")

DILATION


In [6]:
np.unique(mask_result.np)

array([0, 1], dtype=int16)

In [7]:
spam_vol

In [6]:

def realign(skel_f):
    """Realigns skeleton mask to spam
    
    skel_f is a file name of skeleton file"""
    
    # Reads spam and skeleton files

    skel_data = aims.read(skel_f)
    skel_data.np[:] = (skel_data.np > 0).astype(np.int16)

    # Masks skeleton data with dilated spam
    skel_data.np[mask_result.np <= 0] = 0
    print(np.unique(skel_data.np))
    aims.write(skel_data, "/tmp/skel_before.nii.gz")
    
    # Makes realignment
    os.system("ls")
    os.system("AimsMIRegister -r /tmp/spam.nii.gz  -t /tmp/skel_before.nii.gz --dir /tmp/transform.trm")
    # print(out_tr.np)
    
    # Applies the realignment
    os.system("AimsApplyTransform -i /tmp/skel_before.nii.gz -o /tmp/skel_realigned.nii.gz -m /tmp/transform.trm")
    
    # loads realigned file:
    after = aims.read("/tmp/skel_realigned.nii.gz")
    
    return after

In [7]:
# os.system(f"AimsMIRegister -r /tmp/spam.nii.gz  -t /tmp/skel_before.nii.gz --dir /tmp/transform.trm")

In [8]:
np.unique(spam_vol.np)

array([0.        , 0.01639344, 0.03278688, 0.04918033, 0.06557377,
       0.08196721, 0.09836066, 0.1147541 , 0.13114753, 0.14754099,
       0.16393442, 0.18032786, 0.19672132, 0.21311475, 0.22950819,
       0.24590164, 0.26229507, 0.27868852, 0.29508197, 0.3114754 ,
       0.32786885, 0.3442623 , 0.36065573, 0.37704918, 0.39344263,
       0.40983605, 0.4262295 , 0.44262296, 0.45901638, 0.47540984,
       0.4918033 , 0.5081967 , 0.52459013, 0.5409836 , 0.55737704],
      dtype=float32)

In [9]:
np.unique(mask_result.np)

array([0, 1], dtype=int16)

In [10]:
after_all = aims.Volume(mask_result.getSize(), 'S16')
after_all.copyHeaderFrom(mask_result.header())
list_after = []
for skel_f in skel_files[:10]:
    after = aims.Volume(mask_result.getSize(), 'S16')
    after.copyHeaderFrom(mask_result.header())
    after += realign(skel_f)
    after_all += after
    list_after.append(after)

[0 1]
loop_realignment_with_crop.ipynb
loop_realignment_with_crop_MI.ipynb
skel_before.nii.gz
skel_before.nii.gz.minf
test_denis_realignment.ipynb
test_different_alignments.ipynb
test_realignment_with_crop.ipynb
test_realignment_with_crop.py
test_realignment_with_crop_MI.ipynb


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


objective function   : mutual information
optimizer            : powell
Reading channel -1 of reference image
Reference Image       : /tmp/spam.nii.gz
         Pyramid range: 0 - 0
Test      Image       : /tmp/skel_before.nii.gz
         Pyramid range: 0 - 0
Number of Level       : 64
Parzen Mask Size      : 5
Error (Stop Criterion): 0.01
Ref Threshold  ratio  : 0.05
Test Threshold ratio  : 0.1
initial parameter     : (0, 0, 0, 0, 0, 0)
parameter variations  : (0, 0, 0, 0, 0, 0)
Processing Plevel 0(0, 0, 0, 0, 0, 0)(1, 1, 0.596832, 1, 0.596832, 0.502595)
40001090 ticks
Niveau : 0 Popt (0.354706, 1.40921, -0.000793157, -0.0111871, 2.22417e-05, -2.01505e-05)
Volume 0Popt (0.354706, 1.40921, -0.000793157, -0.0111871, 2.22417e-05, -2.01505e-05)
 -Tx 0.354706 -Ty 1.40921 -Tz -0.0111871 -Rx 2.22417e-05 -Ry -2.01505e-05 -Rz -0.000793157
Direct FileName /tmp/transform.trm
Inverse FileName /tmp/spam.nii_TO_skel_before.nii.trm


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


loading direct transformations
Output dimensions: 96, 114, 96
Output voxel size: 2, 2, 2 mm
Resampling carto_volume of S16...   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 %
[0 1]
loop_realignment_with_crop.ipynb
loop_realignment_with_crop_MI.ipynb
skel_before.nii.gz
skel_before.nii.gz.minf
test_denis_realignment.ipynb
test_different_alignments.ipynb
test_realignment_with_crop.ipynb
test_realignment_with_crop.py
test_realignment_with_crop_MI.ipynb


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


objective function   : mutual information
optimizer            : powell
Reading channel -1 of reference image
Reference Image       : /tmp/spam.nii.gz
         Pyramid range: 0 - 0
Test      Image       : /tmp/skel_before.nii.gz
         Pyramid range: 0 - 0
Number of Level       : 64
Parzen Mask Size      : 5
Error (Stop Criterion): 0.01
Ref Threshold  ratio  : 0.05
Test Threshold ratio  : 0.1
initial parameter     : (0, 0, 0, 0, 0, 0)
parameter variations  : (0, 0, 0, 0, 0, 0)
Processing Plevel 0(0, 0, 0, 0, 0, 0)(1, 1, 0.596832, 1, 0.596832, 0.502595)


In [ ]:
np.unique(after_all.np)

array([0, 1, 2, 3, 4, 5, 6], dtype=int16)

In [ ]:
# Visualization
spam = a.loadObject(spam_file)
spam.setPalette("Blues")
spam_after = a.toAObject(after_all)
spam_after.setPalette("Reds")
w = a.createWindow('Sagittal')
w.addObjects(spam)
w.addObjects(spam_after)

observable 0x5dbf32cebe90(N9anatomist7AVolumeIsEE) could not be removed from observer 0x5dbf37d3e518 (N9anatomist8Fusion2DE)


Position : 126, 146.496, 107.777, 0
Position : 98, 182.755, 111.576, 0


In [ ]:
# spam = a.loadObject(spam_file)
# spam.setPalette("Blues")
# list_after_a = [a.toAObject(after) for after in list_after]
# for after in list_after_a:
#     after.setPalette("RED-lfusion")
# w = a.createWindow('Sagittal')
# w.addObjects(list_after_a)
# w.addObjects(spam)

Multitexturing present
function glActiveTexture found.
function glClientActiveTexture found.
function glBlendEquation found.
function glTexImage3D found.
function glMultiTexCoord3f found.
function glBindFramebuffer found.
function glBindRenderbuffer found.
function glFramebufferTexture2D found.
function glGenFramebuffers found.
function glGenRenderbuffers found.
function glFramebufferRenderbuffer found.
function glRenderbufferStorage found.
function glCheckFramebufferStatus found.
function glDeleteRenderbuffers found.
function glDeleteFramebuffers found.
Number of texture units: 4
function glUniform1f found.
function glUniform1i found.
function glUniform4fv found.
function glGetUniformLocation found.
function glMultTransposeMatrixf found.
function glAttachShader found.
function glDetachShader found.
function glCompileShader found.
function glCreateProgram found.
function glCreateShader found.
function glDeleteProgram found.
function glDeleteShader found.
function glGetProgramiv found.


observable 0x5dbf32cebe90(N9anatomist7AVolumeIsEE) could not be removed from observer 0x5dbf37d3e518 (N9anatomist8Fusion2DE)
observable 0x5dbf32cebe90(N9anatomist7AVolumeIsEE) could not be removed from observer 0x5dbf37d3e518 (N9anatomist8Fusion2DE)
observable 0x5dbf32cebe90(N9anatomist7AVolumeIsEE) could not be removed from observer 0x5dbf37d3e518 (N9anatomist8Fusion2DE)
observable 0x5dbf32cebe90(N9anatomist7AVolumeIsEE) could not be removed from observer 0x5dbf37d3e518 (N9anatomist8Fusion2DE)
observable 0x5dbf32cebe90(N9anatomist7AVolumeIsEE) could not be removed from observer 0x5dbf37d3e518 (N9anatomist8Fusion2DE)
observable 0x5dbf32cebe90(N9anatomist7AVolumeIsEE) could not be removed from observer 0x5dbf37d3e518 (N9anatomist8Fusion2DE)
observable 0x5dbf32cebe90(N9anatomist7AVolumeIsEE) could not be removed from observer 0x5dbf37d3e518 (N9anatomist8Fusion2DE)
observable 0x5dbf32cebe90(N9anatomist7AVolumeIsEE) could not be removed from observer 0x5dbf37d3e518 (N9anatomist8Fusion2DE)


Position : 96, 222.333, 178.333, 0
Position : 124, 143.219, 107.471, 0
Position : 124, 142.259, 106.832, 0
